# Batch Transform for Explanations

In [ ]:
import boto3
from pathlib import Path
import sagemaker
from sagemaker.transformer import Transformer

from package import config, utils

In [ ]:
current_folder = utils.get_current_folder(globals())
sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client('sagemaker')

In [ ]:
def get_latest_model(name_contains):
    response = sagemaker_client.list_models(
        NameContains=name_contains
    )
    models = response['Models']
    assert len(models) > 0, "Couldn't find any models with '{}' in name.".format(name_contains)
    latest_model = models[0]['ModelName']
    return latest_model

In [ ]:
latest_model = get_latest_model(config.STACK_NAME)
job_name = latest_model

In [ ]:
batch_explainer = Transformer(
    model_name=latest_model,
    instance_count=1,
    instance_type='ml.c5.xlarge',
    strategy='SingleRecord',
    assemble_with='Line',
    output_path='s3://' + str(Path(config.S3_BUCKET, 'explanations', job_name)) + '/',
    accept='application/json',
    base_transform_job_name=latest_model,
    sagemaker_session=sagemaker_session
)

In [ ]:
entities = [
    'data',
    'features',
    'prediction',
    'explanation_shap_values',
    'explanation_shap_interaction_values'
]
batch_explainer.transform(
    data='s3://' + str(Path(config.S3_BUCKET, config.DATASETS_S3_PREFIX, 'data_test')) + '/',
    content_type="application/json; entities={}".format(",".join(entities)),
    split_type='Line',
    wait=True
)

## Next Stage

Up next we'll develop a dashboard for explanations using Amazon SageMaker and Streamlit.

[Click here to continue.](./5_dashboard.ipynb)